#### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from sklearn.decomposition import TruncatedSVD


/Users/susanliang/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
qs_ranking = pd.read_csv('./young_company/qs_rank_2024.csv', index_col=0)
different_rows = pd.read_csv('institutions_full.csv', header= None)
different_rows = different_rows.rename(columns={0: "index", 1: "institution_name"})


#### University Mapping

##### this section first clean the QS university table and then map the university to the standard QS university name.

In [3]:
raw_rank = pd.read_excel("2024 QS World University Rankings 1.2 (For qs.com) copy.xlsx")
raw_rank.columns = [None] * raw_rank.shape[1]
raw_rank = raw_rank.drop(index=[0, 1])
raw_rank = raw_rank.reset_index()
new_header = raw_rank.iloc[0] 
raw_rank = raw_rank[1:] 
raw_rank.columns = new_header 
raw_rank = raw_rank.drop(columns=[2, 'rank display2','location code'])
raw_rank = raw_rank.rename(columns={"rank display": "rank_2024"})
raw_rank = raw_rank.iloc[:, :3]
raw_rank['institution'] = raw_rank['institution'].str.replace(r'\s*\(.*\)', '', regex=True)
raw_rank = raw_rank.replace(r'[=+]', '', regex=True)
raw_rank['rank_2024'] = raw_rank['rank_2024'].astype(str)
def convert_to_numeric(value):
    if '-' in value:
        start, end = map(int, value.split('-'))
        return (start + end) / 2
    else:
        return int(value)

raw_rank['rank_2024'] = raw_rank['rank_2024'].apply(convert_to_numeric).astype(int)
qs_ranking = raw_rank

In [ ]:
different_rows['Best Match'] = None
different_rows = different_rows.astype(str)
different_rows = different_rows[different_rows['Best Match'] == 'None']

sample_university_names = different_rows[different_rows['Best Match'] == 'None']['institution_name'].tolist()
qs_standard_names = qs_ranking['institution'].tolist()
# Load the Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
qs_embeddings = model.encode(qs_standard_names, convert_to_tensor=True) # create embeddings for QS standard names

# Function to find the best match for each name based on cosine similarity
def match_with_embeddings(name, qs_names, qs_embeddings, threshold=0.6):
    name_embedding = model.encode(name, convert_to_tensor=True)
    # Compute cosine similarity between the name and each QS name
    similarities = util.cos_sim(name_embedding, qs_embeddings)[0]
    best_score = torch.max(similarities).item()
    best_match_idx = torch.argmax(similarities).item()
    best_match = qs_names[best_match_idx] if best_score >= threshold else None
    return best_match

semantic_matches = {}
for name in tqdm(sample_university_names, desc="Matching university names"):
    semantic_matches[name] = match_with_embeddings(name, qs_standard_names, qs_embeddings)

print(semantic_matches)

different_rows = different_rows.astype(str)
different_rows = different_rows[different_rows['Best Match'] != 'None']

matches_df = pd.DataFrame.from_dict(semantic_matches, orient='index', columns=['Best Match'])
matches_df.index.name = 'institution_name' 
matches_df.reset_index(inplace=True)
# matches_df.to_excel('BERT_matches_dfv2.xlsx')
matches_df = pd.DataFrame.from_dict(semantic_matches, orient='index', columns=['Best Match'])
matches_df.index.name = 'institution_name' 
matches_df.reset_index(inplace=True)

##### this section generate SQL statements which can be directly copy-paste into mysql. The mapping output version currently being used for mysql scripts are further being revised manually to ensure there is no meaningless mapping. However, for the sake of workflow, the mapping result is directly translated into mysql code without further manual verification.

In [ ]:
update_statements = []
matches_df = matches_df.astype(str)
for _, row in matches_df.iterrows():
    new_name = row['Best Match'].replace("'", "''")  # Handle single quotes
    old_name = row['institution_name'].replace("'", "''")   # Handle single quotes
    sql = f"UPDATE young_company.institutions SET institution_name = '{new_name}' WHERE institution_name = '{old_name}';"
    update_statements.append(sql)

sql_updates = "\n".join(update_statements)
print("Generated SQL UPDATE statements:")
print(sql_updates)

#### University Program Mapping

##### this section applys topic generation and top modelling to the university study programs of the founders.

In [4]:
df = pd.read_csv('programs_founder.csv', header=None)
df = df.rename(columns={0: "index", 1: "program_name"})

# Step 1: Text vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['program_name'])

# Step 2: Dimensionality reduction for speedup
# Reducing TF-IDF dimensionality with TruncatedSVD
svd = TruncatedSVD(n_components=100, random_state=42)
X_reduced = svd.fit_transform(X)

# Step 3: Clustering with reduced dimensions
num_clusters = 50
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_reduced)

sample_clusters = np.random.choice(df['Cluster'].unique(), 10, replace=False)

for cluster in sample_clusters:
    print(f"\nCluster {cluster + 1}:")
    print(df[df['Cluster'] == cluster]['program_name'].sample(3, random_state=42).values)

/Users/susanliang/opt/anaconda3/envs/nlp/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)



Cluster 6:
['Mathematics' 'Mathematics And Computer Science'
 'Engineering Mathematics']

Cluster 45:
['Industrial And Management Engineering'
 'Econometrics, Industrial Economics' 'Industrial Design, Daap']

Cluster 7:
['Engineering, Applied Mathematics'
 'Bachelor Of Applied Science Computer Engineering'
 'Applied Economics And Management']

Cluster 22:
['Physics, Math' 'Math And Physics' 'Chinese And Math']

Cluster 11:
['Applied Operations Research'
 'Operations Research, Financial Engineering'
 'Operations Research And Industrial Engineering']

Cluster 48:
['Information Systems And Tools For New Media'
 'Business Honors And Management Information Systems, Elements Of Computing Certificate'
 'Information Management And Systems']

Cluster 26:
['International Studies, Focus On Brazil / Portuguese Language'
 'International Baccalaureate' 'International Middle School']

Cluster 34:
['Economics And Finance, Minor In Statistics'
 'Honours Mathematics, Statistics Major' 'Statistics, Math

##### We take the 50 cluster output from above to chatGPT using GPT -4o model with the prompts as follows:

`create 15 names of those 50 clusters, such as computer science, mathemetics, .... that will be further cosine similarity friendly`

##### Then, I create a excel table from the GenAI tag generation result

##### Now, I use LLM transformer to tag the university study programs into those 15 tags

In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
programs_df = df[['index', 'program_name']]  
categories_df = pd.read_excel('program_category_output.xlsx') 
category_embeddings = model.encode(categories_df['Program_Category'].tolist(), convert_to_tensor=True)
assigned_categories = []
for program_name in tqdm(programs_df['program_name'], desc="Classifying programs"):
    program_embedding = model.encode(program_name, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(program_embedding, category_embeddings)
    best_match_idx = int(similarities.argmax())
    best_category = categories_df.iloc[best_match_idx]['Program_Category']
    assigned_categories.append(best_category)
programs_df['category'] = assigned_categories

Classifying programs: 100%|██████████| 3388/3388 [00:51<00:00, 65.71it/s]
/var/folders/wr/956qfj8n0nd7qf20gnbgf5qw0000gn/T/ipykernel_45792/4007075045.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  programs_df['category'] = assigned_categories


In [13]:
# programs_df.to_csv('mapping_program_category.csv')